# Object Detection Example

## Introduction

In this notebook, we'll walk through a detailed example of how you can use Valor to evaluate object detections made on [the COCO Panoptic dataset](https://cocodataset.org/#home). We'll use Ultralytics' `YOLOv8` model to predict what objects exist in various COCO photographs and compare performance between bounding box and image segmentation results.

For a conceptual introduction to Valor, [check out our project overview](https://striveworks.github.io/valor/). For a higher-level example notebook, [check out our "Getting Started" notebook](https://github.com/Striveworks/valor/blob/main/examples/getting_started.ipynb).

Before using this notebook, please ensure that the Valor service is running on your machine (for start-up instructions, [click here](https://striveworks.github.io/valor/getting_started/)). To connect to a non-local instance of Valor, update `client = Client("http://0.0.0.0:8000")` in the first code block to point to the correct URL.

## Defining Our Datasets

We start by fetching our dataset and uploading it to Valor.

In [1]:
import ultralytics

from tqdm import tqdm
from pathlib import Path
import pandas as pd

from valor import Client, Model, Annotation, Label, connect
from valor.enums import TaskType, AnnotationType
from valor.viz import create_combined_segmentation_mask

# connect to Valor API
connect("http://localhost:8000")
client = Client()

Successfully connected to host at http://localhost:8000/


/var/folders/c_/vxjvkhy543l66mrkrtfrb56c0000gn/T/ipykernel_97405/3282455273.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


The modules included in `./integrations` are helper modules that demonstrate how to ingest datasets and model inferences into Valor. The depth of each integration varies depending on the use case. 

The `coco_integration` is designed to download, extract, and upload all in one command as you are starting off with all the the data. 

The `yolo_integration` is much simpler; it is a collection of parser functions that convert YOLO model results into Valor types.

In [2]:
import integrations.coco_integration as coco
import integrations.yolo_integration as yolo

# Defining Our Dataset

This block utilizes `create_dataset_from_coco_panoptic` from `integrations/coco_integration.py` to download, extract, and upload the COCO Panoptic validation dataset to Valor.

In [3]:
# create the dataset in Valor
valor_dataset = coco.create_dataset_from_coco_panoptic(
    destination=Path("./").absolute().parent / Path("coco"),
    limit=5,
)

coco already exists at /Users/nthorlind/git/sw/valor/examples/coco!


Uploading: 100%|██████████| 5/5 [00:00<00:00,  8.85it/s]


## Defining Our Model

With our `Dataset` in Valor, we're ready to create our `Model` object and add `Predictions` to it.

In [4]:
# define the model in Valor. note that we can use any name we'd like
valor_model_bbox = Model.create("yolov8n-box")
valor_model_seg = Model.create("yolov8n-seg")

inference_engine = ultralytics.YOLO(f"yolov8n-seg.pt")

for datum in tqdm(valor_dataset.get_datums()):

    image = coco.download_image(datum)

    results = inference_engine(image, verbose=False)

    # convert result into Valor Bounding Box prediction
    bbox_prediction = yolo.parse_detection_into_bounding_box(
        results,            # raw inference
        datum=datum,        # valor datum
        label_key='name',   # label_key override
    )

    # convert result into Valor Raster prediction
    raster_prediction = yolo.parse_detection_into_raster(
        results,            # raw inference
        datum=datum,        # valor datum
        label_key='name',   # label_key override
    )

    # add predictions to the model
    valor_model_bbox.add_prediction(valor_dataset, bbox_prediction)
    valor_model_seg.add_prediction(valor_dataset, raster_prediction)

  0%|          | 0/5 [00:00<?, ?it/s][W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
/opt/homebrew/anaconda3/envs/velour_api_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 80%|████████  | 4/5 [00:05<00:01,  1.02s/it]

## Exploring Our Dataset

Before we evaluate our results, let's check out what's stored in Valor. Below, we show an example of a COCO image (in this case, the image we added using UID '139').

In [ ]:
groundtruth_139 = valor_dataset.get_groundtruth('139')
assert groundtruth_139
coco.download_image(groundtruth_139.datum)

Next, we visualize multiple segmentation masks to show all of the objects we want to be able to detect.

In [ ]:
# TODO fix 
instance_mask, instance_legend = create_combined_segmentation_mask(
    groundtruth_139, 
    label_key="name",
    task_type=TaskType.OBJECT_DETECTION,
)

instance_mask

In [ ]:
# print the color code for the above segmentations
for k, v in instance_legend.items():
    print(k)
    display(v)

## Evaluating Performance

With our `Dataset` and `Model` defined, we're ready to evaluate our performance and display the results. Note that we use the `wait_for_completion` method since all evaluations run as background tasks; this method ensures that the evaluation finishes before we display the results.

Sometimes, we may only want to calculate metrics for a subset of our data (i.e., we may only want to see how well our model performed at a specific type of detection). To accomplish this task, we can use the `filters` parameter of `evaluation_detection` to specify what types of data to evaluate performance for.

We will be running and comparing two different evaluations investigating the performance difference of YOLOv8's bounding box and raster outputs.

In [ ]:
# bounding box evaluation
eval_bbox = valor_model_bbox.evaluate_detection(
    valor_dataset,
    filter_by=[
        Label.key == "name",
    ],
)
eval_bbox.wait_for_completion()
eval_bbox.metrics

In [ ]:
# raster evaluation
eval_raster = valor_model_seg.evaluate_detection(
    valor_dataset,
    filter_by=[
        Label.key == "name",
    ]
)
eval_raster.wait_for_completion()
eval_raster.metrics

We can compare performance by comparing our results in pandas dataframe.

In [ ]:

bdf = eval_bbox.to_dataframe(("annotation type", "bbox"))
rdf = eval_raster.to_dataframe(("annotation type", "raster"))
pd.concat([bdf, rdf], axis=1, names=["bbox", "raster"])

## Evaluating based on object size.

Filters are not limited to annotation type and label keys as shown above. We can also define filters for a pixel-wise geometric area that will help us test the performance of objects that fall within certain size ranges.

In [ ]:
lower_bound = 30000
upper_bound = 100000

### Small Object Evaluation

In [ ]:
# bounding box evaluation
eval_bbox_small = valor_model_bbox.evaluate_detection(
    valor_dataset,
    filter_by=[
        Label.key == "name",
        Annotation.bounding_box.area < lower_bound,
    ],
)
eval_bbox_small.wait_for_completion()

In [ ]:
# raster evaluation
eval_raster_small = valor_model_seg.evaluate_detection(
    valor_dataset,
    filter_by=[
        Label.key == "name",
        Annotation.raster.area < lower_bound,
    ]
)
eval_raster_small.wait_for_completion()

In [ ]:
bbox_df = eval_bbox_small.to_dataframe(("annotation type", "bbox"))
raster_df = eval_raster_small.to_dataframe(("annotation type", "raster"))
pd.concat([bbox_df, raster_df], axis=1, names=["bbox", "raster"])

### Mid-sized Object Evaluation

In [ ]:
# bounding box evaluation
eval_bbox_mid = valor_model_bbox.evaluate_detection(
    valor_dataset,
    filter_by=[
        Label.key == "name",
        Annotation.bounding_box.area >= lower_bound,
        Annotation.bounding_box.area <= upper_bound,
    ],
)
eval_bbox_mid.wait_for_completion()

In [ ]:
# raster evaluation
eval_raster_mid = valor_model_seg.evaluate_detection(
    valor_dataset,
    filter_by=[
        Label.key == "name",
        Annotation.raster.area >= lower_bound,
        Annotation.raster.area <= upper_bound,
    ]
)
eval_raster_mid.wait_for_completion()

In [ ]:
bbox_df = eval_bbox_mid.to_dataframe(("annotation type", "bbox"))
raster_df = eval_raster_mid.to_dataframe(("annotation type", "raster"))
pd.concat([bbox_df, raster_df], axis=1, names=["bbox", "raster"])

### Large Object Evaluation

In [ ]:
# bounding box evaluation
eval_bbox_large = valor_model_bbox.evaluate_detection(
    valor_dataset,
    filter_by=[
        Label.key == "name",
        Annotation.bounding_box.area > upper_bound,
    ],
)
eval_bbox_large.wait_for_completion()

In [ ]:
# raster evaluation
eval_raster_large = valor_model_seg.evaluate_detection(
    valor_dataset,
    filter_by=[
        Label.key == "name",
        Annotation.raster.area > upper_bound,
    ]
)
eval_raster_large.wait_for_completion()

In [ ]:
bbox_df = eval_bbox_large.to_dataframe(("annotation type", "bbox"))
raster_df = eval_raster_large.to_dataframe(("annotation type", "raster"))
pd.concat([bbox_df, raster_df], axis=1, names=["bbox", "raster"])